### This Dev notebook builds simple XGB model to predict baby weight

### Outline:
1. Ingest data using BigQuery API.
2. Clean the data.
3. Build model.

In [1]:
project_name = 'My First Project'
project_id = 'quantum-keep-360100'
regionn = 'us-central1'

In [2]:
import pandas as pd
import xgboost as xgb
import numpy as np
import time
import pickle
import os

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from google.cloud import bigquery, storage

# xgb.__version__
os.chdir('/home/jupyter/projects_gcp')

In [3]:
time0 = time.time()

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
display(df.shape, df.head())

(10000, 5)

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,9.312326,False,28,1,40.0
1,7.749249,True,30,1,40.0
2,7.394304,True,27,1,39.0
3,6.750554,False,40,1,41.0
4,8.377566,True,24,1,38.0


In [4]:
display(df.describe())
display(df.is_male.value_counts())

,weight_pounds,mother_age,plurality,gestation_weeks
count,9984.000000,10000.000000,10000.000000,9935.000000
mean,7.207753,27.394900,1.032900,38.545848
std,1.325683,6.142551,0.187673,2.561521
min,0.639341,13.000000,1.000000,17.000000
25%,6.563162,23.000000,1.000000,38.000000
50%,7.312733,27.000000,1.000000,39.000000
75%,8.035849,32.000000,1.000000,40.000000
max,12.500210,47.000000,4.000000,47.000000


True     5129
False    4871
Name: is_male, dtype: int64

In [5]:
df['is_male'] = df['is_male'].astype(int)
df = df.dropna()
df = shuffle(df, random_state=2)

labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
x,y = data,labels
X_train, X_test, y_train, y_test = train_test_split(x,y)

display(X_train.shape, X_train.head(), X_test.shape, y_train.shape)

(7442, 4)

,is_male,mother_age,plurality,gestation_weeks
4685,0,35,1,38.0
5336,1,46,1,40.0
7838,1,24,1,37.0
63,0,28,1,39.0
962,1,22,1,41.0


(2481, 4)

(7442,)

In [6]:
time1 = time.time()
model = LinearRegression()

model.fit(X_train, y_train)
model_copied = model

print(time.time()-time1)

0.021043777465820312


In [7]:
y_pred = model.predict(x_test)

for i in range(5):
    print('Predicted weight: ', y_pred[i])
    print('Actual weight: ', y_test.iloc[i])
    print()
    
print('train rmse: ', np.sqrt(mean_squared_error(y_train, model.predict(x_train))))
print('test rmse: ', np.sqrt(mean_squared_error(y_test, model.predict(x_test))))

Predicted weight:  8.085394
Actual weight:  6.75055446244

Predicted weight:  6.548306
Actual weight:  7.85727501768

Predicted weight:  7.443437
Actual weight:  6.87621795178

Predicted weight:  7.2450414
Actual weight:  7.87491199864

Predicted weight:  7.989084
Actual weight:  7.5398093604

train rmse:  1.0628292708888534
test rmse:  1.0402076726258331
